CIP Project Clean and Enrich A.ipynb <br>
Author: Esin Handenur Isik

# Clean and Enrich Box Office Mojo Data Set
File has been synthetically made dirty. 

## 1. Cleaning processes:

### Already present impurities:

1.1. remove currency sign and commas from Budget and Revenue column <br>
1.2. change dataypes of Budget and Revenue and Rank

### Synthetically added impurities:

1.3. Put ranking in correct order <br> 
1.4. Correct the release years <br>
     + Remove movies released after 2017<br>
1.5. Detect unrealistic and inaccurate budget information and change to NaN <br>
1.6. Remove incorrect characters from titles <br>

## 2. Enrichment processes:

2.1. Create column with lifetime gross revenue to budget ratio <br>
2.2. Create column for calculated Lifetime Gross Profit <br>
2.3. Create ranking based on Liftetime Gross Profit

##### Load impure data A_stage2.csv:

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("../Data/A_stage2.csv")
df

## 1. Cleaning Process
### Clean present impurities
#### 1.1. Remove currency sign and commas from Budget and Revenue column
#### 1.2. Change dataypes of Budget and Revenue and Rank

In [ ]:
df.Budget = df.Budget.str.replace("$", "", regex = True).replace(",", "", regex = True)
df.Budget = df.Budget.str.replace(".", "", regex = True).fillna(0).astype(int) # fill NaN to convert data type
df["Lifetime Gross"] = df["Lifetime Gross"].str.replace("$", "", regex = True).replace(",", "", regex = True).astype(int)
df.Rank = df.Rank.astype(int)
df

In [ ]:
print(type(df.Budget.loc[0])) #values are now integers
print(type(df['Lifetime Gross'].loc[0]))
print(type(df.Rank.loc[0]))

#### 1.3. Put ranking in correct order

In [ ]:
df = df.sort_values("Rank", ignore_index = True)
df

#### 1.4. Correct the release years

In [ ]:
df.Year = df.Year.astype(str).str.split(pat = ".") # pandas split to split year and random month info
df.Year = df.Year.str.get(0).astype(int) # extract first part of split --> Year
df

In [ ]:
print(type(df.Year.loc[0])) # Years are integers

#### Delete all movies released after 2017 to match with Data Source B and C:

In [ ]:
df = df.drop(df[df.Year > 2017].index)
#Rewrite Rank to avoid gaps:
df = df.reset_index(drop= True) # remove index with gaps 
df = df.reset_index() # adds old index as column, can be used to replace ranking
df

In [ ]:
df.Rank = df.index + 1 # replace rank with old index column, +1 to start Rank at 1
df = df.drop("index",axis =1) 
df

#### 1.5. Detect unrealistic and inaccurate budget information and change to NaN

In [ ]:
df.loc[df.Budget < 1000, 'Budget'] = np.nan # converts all previously NaN's set 0 and inaccurate budgets to NaN
df.Budget = df.Budget.astype("Int64") # Optional: np.nan casts column as float, convert back into int
df

#### 1.6. Remove incorrect characters from titles

In [ ]:
# Titles that contain a #:
print("Titles that contain a #: " + str(len(df[df.Title.str.contains('#')])))
df[df.Title.str.contains('#')]

In [ ]:
#Delete all #:
df.Title = df.Title.str.replace("#", "", regex = False)

In [ ]:
print(len(df[df.Title.astype(str).str.contains('#')])) # check if all # were removed
df.Title.loc[30] # E.g., Title at index 30 used to be "Harry Potter and# the Sorcerer's Stone"

## 2. Enrichment Process
#### 2.1. Create column with lifetime gross revenue to budget ratio

In [ ]:
ratios = []

for x,y in zip(df["Lifetime Gross"], df.Budget): # use for loop to calculate ratio and to be able to handle NaN exeptions
    try:
        ratio = round(x / y, 2) # round to 2 decimals 
        ratios.append(ratio)
    except:
        ratio = np.nan
        ratios.append(ratio)

print(len(ratios))
df["R:B Ratio"] = pd.Series(ratios) # add list of ratios as column "Ratio" to df
df

#### 2.2. Create column for calculated Lifetime Gross Profit

In [ ]:
profits = []

for x,y in zip(df["Lifetime Gross"], df.Budget): # use for loop to be able to handle NaN exceptions
    try:
        profit = x - y # obtain profit by subtracting the movie's budget from its' revenue
        profits.append(profit)
    except:
        profit = np.nan
        profits.append(profit)

print(len(profits))
df["Gross Profit"] = pd.Series(profits) # add list of profits as column "Gross Profit" to df
df

#### 2.3. Create ranking based on Liftetime Gross Profit

In [ ]:
numNA = df["Gross Profit"].isna().sum()
print(numNA)

In [ ]:
# create pd.Series of sorted Gross Profit values, descending, place all NA last:
profit_ranking = df["Gross Profit"].sort_values(ascending = False, na_position = "last") 

dflen1 = len(df)+1
ranking = []
for rank, p in zip(range(1,dflen1), profit_ranking): #loop over the Series to create new ranking
    if rank < (dflen1-numNA):
        ranking.append(rank)
    else: 
        rank = pd.NA
        ranking.append(rank)


print(len(ranking))

#ranking is sorted according to the sorted Gross Profit column, match df to same order to append ranking
df = df.sort_values("Gross Profit",ascending = False, na_position = "last", ignore_index = True)

df["Profit Rank"] = pd.Series(ranking) # add list of profits as column "Gross Profit" to df
df

### Final Adjustments:
#### Reorder table according to Rvenue ranking and reset index:

In [ ]:
df = df.sort_values("Rank", ignore_index = True)
df

#### Rename Revenue ranking and reorder columns:

In [ ]:
df.rename(columns={'Rank':'Revenue Rank'}, inplace=True) # rename "Rank" to differenciate from "Profit Rank"
df.rename(columns={'Lifetime Gross':'Gross Revenue'}, inplace=True)
df

In [ ]:
df = df.reindex(columns = ['Revenue Rank', 'Profit Rank', 'Title', 'Budget', 'Gross Revenue', 'R:B Ratio',
       'Gross Profit', 'Year']) # reorder columns
df

#### Inspect and correct df.Title to prepare as Primary Key in Table in MariaDB:

In [ ]:
dfdupl = df[df.duplicated("Title", keep = False)]
dfdupl 
# it shows that there are 8 duplicated values in the Title column. 
#These are re-released movies with the same title

In [ ]:
# It is possible to differenciate those titles by concatenating them with their release years
def correct_dupl(dataframe):
    df = dataframe

    dupl_titles = df.Title.duplicated(keep=False) #Select the duplicated titles

    df.loc[dupl_titles, "Title"] = (df.loc[dupl_titles, "Title"] + " (" + 
                                    df.loc[dupl_titles, "Year"].astype(str) + ")") # re-define titles with string concatenation

    return df

df = correct_dupl(df)
df

In [ ]:
# Check a duplicate: Beauty and the Beast at index 10:
df.Title.loc[10]

#### Export A_stage3.csv:

In [ ]:
df.to_csv("../Data/A_stage3.csv", index = False)